# Generador

In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from pathlib import Path
from detection_helper import create_DF
from matplotlib import pyplot as plt
import albumentations as A
from PIL import Image
import matplotlib.patches as patches
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\bruno\AppData\Local\Programs\Python\Python311\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
train_files = list(Path("Dataset/train/").glob("**/*txt"))
val_files = list(Path("Dataset/validation/").glob("**/*txt"))

In [3]:
df_train = create_DF(train_files,Path("Dataset/train/Apple_Grape_Cantaloupe_Watermelon_Pomegranate_Pineapple_Peach_Mango_Banana_Lemon_Tomato_Strawberry_Pear/"))

In [7]:
df_train.to_hdf("df_train.hdf", key="df")

In [8]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, df, albumentator, anchor_boxes, IoU_alpha=0.5, batch_size=32, crop_size=[512,512], grid_size=[32,32]):
        'Initialization'
        self.df = df.copy()
        self.albumentator = albumentator
        self.anchor_boxes = anchor_boxes
        self.cant_anchor_boxes = len(self.anchor_boxes)
        self.IoU_alpha = IoU_alpha
        self.batch_size = batch_size
        self.crop_size = [512,512]
        self.grid_size = [32,32]
        self.classes = self.df["label"].unique()
        self.classes.sort()
        self.classes_idx = {clase:idx for idx,clase in enumerate(self.classes)}
        self.cant_classes = len(self.classes)
        self.list_IDs = self.df["image_name"].unique()
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        print(list_IDs_temp)
        return 1,1
#         # Initialization
#         X = np.empty((self.batch_size, *self.dim, self.n_channels))
#         y = np.empty((self.batch_size), dtype=int)

#         # Generate data
#         for i, ID in enumerate(list_IDs_temp):
#             # Store sample
#             X[i,] = np.load('data/' + ID + '.npy')

#             # Store class
#             y[i] = self.labels[ID]

#         return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [9]:
with open("anchor_boxes.pkl", "rb") as f:
    anchor_boxes = pickle.load(f)
cant_anchor_boxes = len(anchor_boxes)

In [10]:
transform = A.Compose([
    A.RandomCrop(width=512, height=512),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='coco', min_area=1024, min_visibility=0.1))

In [11]:
gen = DataGenerator(df_train,transform,anchor_boxes=anchor_boxes)

In [12]:
len(gen)

224

In [15]:
gen[0]

['76d74e8afd19313e', '20d53a3143fa41b8', '9bb208229bfd7175', 'ef16b1f81560c0bd', 'fdc1fc0d417075ea', '11a15fcf7426166d', '655416f183f629ce', '48fd868dbe1f524a', 'f9c7d16af494c212', '00d1a44121b1b883', '07d482fa22194c70', '1f76ac5e32b66c76', 'e1805a3977f44cfb', '02fb78b224bc4418', 'd80173d01625b2fa', '1ba8191d864987f4', 'ddf1c169ae6805f5', 'ae2931c5820d4f3d', '8c981905e54c01c0', '0dc8a1ebc366a5c1', '8a8d9e65261f0cc3', 'f9a6f4038890276a', '00730e60ab276b06', '87e88ea71a9c8a71', 'bb2dd21db373a147', '811523dbfba99205', '0e5b1f1a294fad14', 'd0a8c3d2707ede8e', '2c7ed825d739673d', '32229d0bed0d5181', 'a3a598da5c9e1c2c', '4a9d7d2790403b67']


(1, 1)